In [48]:
import requests
import datetime

def populate_list(number_days_window):
    populated_list = []
    #all_days = requests.get('https://api.covid19api.com/dayone/country/argentina/status/confirmed/live')
    #all_days = requests.get('https://api.covid19api.com/dayone/country/argentina/status/confirmed/live', verify=False)
    all_days = requests.get('https://api.covid19api.com/total/country/argentina')
    first=True
    for day in all_days.json():
        date_time_obj = datetime.datetime.strptime(day['Date'][:10], '%Y-%m-%d')
        if first:
            ordinal_first = date_time_obj.toordinal()
        n = date_time_obj.toordinal() - ordinal_first
        delta = 0
        fatality_rate = 0
        if n>number_days_window:
            delta = day['Confirmed'] - populated_list[n-number_days_window][1] 
        first=False
        if day['Confirmed'] > 0:
            fatality_rate = day['Deaths']/day['Confirmed']
        populated_list.append((n, day['Active'], day['Confirmed'], delta, fatality_rate, date_time_obj))
    return populated_list

In [36]:
import pandas as pd
import numpy as np

# Si fatality rate sube, se esta subtesteando, hay mas casos de lo que se cree.

def process_means(number_days_window, jump_days, total_lapse_day):
    my_populated_list = populate_list(number_days_window)
    means_list = []
    fatality_rate = []
    active_cases = []

    df = pd.DataFrame (my_populated_list,columns=['day-number', 'active', 'cases-count','delta', 'fatality_rate','date'])
    
    for n in range(total_lapse_day,0,-jump_days):
        try:
            mean = round(df[-(n):-(n-(jump_days))]['delta'].mean())
            active = round(df[-(n):-(n-(jump_days))]['active'].mean())
        except:
            mean = round(df[-(n):-(n-(jump_days-1))]['delta'].mean())
            active = round(df[-(n):-(n-(jump_days-1))]['active'].mean())
        means_list.append(mean)
        fatality_rate.append(df[-n:-(n-1)]['fatality_rate'].mean())
        active_cases.append(active)
    return means_list, fatality_rate, active_cases



In [45]:
a = range(5)

#print(process_means(30,14,70))

#means_list, fatality_rate, active_cases = process_means(30,14,70)
from scipy.stats import linregress
for params in [process_means(30,14,70), process_means(14,14,70), process_means(10,14,70), process_means(7,14,70), process_means(5,14,70)]:
    means_list, fatality_rate, active_cases = params
    for result in [means_list, fatality_rate, active_cases]:
        print(result)
        print(linregress(a, result))
    print ("...----...")

[1065444, 1176682, 1284054, 1375623, 1479228]
LinregressResult(slope=102650.90000000001, intercept=1070904.4, rvalue=0.9994460538180151, pvalue=1.5649451764737468e-05, stderr=1973.4739986463608)
[0.026614792795943348, 0.027077840031949702, 0.02711173326873565, 0.027277400864651306, 0.0270573227228693]
LinregressResult(slope=0.000108462068655351, intercept=0.026810893799519155, rvalue=0.6953612113665375, pvalue=0.19235170831815573, stderr=6.471880891532188e-05)
[154612, 143024, 130242, 128312, 143853]
LinregressResult(slope=-3623.0, intercept=147254.6, rvalue=-0.5288469182843798, pvalue=0.3595077561200439, stderr=3356.9181163680473)
...----...
[1043357, 1186444, 1297894, 1390848, 1483833]
LinregressResult(slope=108535.6, intercept=1063404.0, rvalue=0.9953337595066747, pvalue=0.0003823669289569103, stderr=6074.830766367023)
[0.026614792795943348, 0.027077840031949702, 0.02711173326873565, 0.027277400864651306, 0.0270573227228693]
LinregressResult(slope=0.000108462068655351, intercept=0.0

Si fatality_rate > fatality_rate_mean:
    Yellow Warning Casos pueden ser mayor de lo reflejado pues pueden que estén subtesteando.

Si fatality_rate > fatality_rate_mean y active_cases y means_list en crecimiento:

    Rojo Warning 

means_list, fatality_rate, active_cases


In [4]:
from random import choice
from experta import *


class Light(Fact):
    """Info about the traffic light."""
    pass


class RobotCrossStreet(KnowledgeEngine):
    @Rule(Light(color='green'))
    def green_light(self):
        print("Walk")

    @Rule(Light(color='red'))
    def red_light(self):
        print("Don't walk")

    @Rule(AS.light << Light(color=L('yellow') | L('blinking-yellow')))
    def cautious(self, light):
        print("Be cautious because light is", light["color"])


engine1 = RobotCrossStreet()
engine1.reset()
engine1.declare(Light(color=choice(['green', 'yellow', 'blinking-yellow', 'red'])))
engine1.run()


Be cautious because light is blinking-yellow
